In [151]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Function
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import random
import matplotlib.pyplot as plt
import warnings
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
import scipy.io
import pdb

import os
from PIL import Image
from torchvision.datasets import VisionDataset
from torchvision.datasets.utils import download_and_extract_archive

warnings.filterwarnings('ignore')

def classification_acc(y_pred, y_true):
    y_pred_tag = torch.softmax(y_pred, dim=1).argmax(dim=1)
    y_true_tag = torch.softmax(y_true, dim=1).argmax(dim=1)

    correct_results_sum = (y_pred_tag == y_true_tag).float().sum()
    acc = correct_results_sum/float(y_true.size(0))*100
    return acc

def domain_acc(y_pred, y_true):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_true).sum().float()
    acc = correct_results_sum/y_true.shape[0]*100
    
    return acc
    
## Dataset class for training
class SimpleDataset(Dataset):
    
    def __init__(self, X_data, y_data):
        #pdb.set_trace()
        self.X_data = torch.unsqueeze(X_data, dim=1)
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

def convert_X_y_mat2py(X, y):
    
    X = X.transpose()
    y = y.squeeze()
    y = y - 1
    
    return X, y

In [152]:
# Third code block - data loading
BATCH_SIZE = 64
test_set_name = 'CWRU' # 'MFPT', 'PU', 'XJTU_SY', 'PRONOSTIA-FEMTO', 'NBSWT'
datasets_names = ['CWRU', 'MFPT', 'PU', 'XJTU_SY', 'PRONOSTIA-FEMTO', 'NBSWT']
data_path = 'E:\\data\\papers\\zero_fault_shot_learning\\datasets'
processed_data = scipy.io.loadmat(data_path + '\\processed_data.mat')

X_training = processed_data["X_training"]
y_training = processed_data["y_training"]
X_training, y_training = convert_X_y_mat2py(X_training, y_training)

X_test = processed_data["X_test"]
y_test = processed_data["y_test"]
X_test, y_test = convert_X_y_mat2py(X_test, y_test)

X_training, y_training = shuffle(X_training, y_training, random_state=0)
X_training, X_val, y_training, y_val = train_test_split(X_training, y_training, 
                                                        test_size=0.3, random_state=42)

training_data = SimpleDataset(torch.FloatTensor(X_training), torch.FloatTensor(y_training))
val_data = SimpleDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
test_data = SimpleDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))

## Creating dataloaders
training_loader = DataLoader(dataset=training_data, batch_size=BATCH_SIZE, shuffle=False)
val_loader = DataLoader(dataset=val_data, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False)

print("Trainning / Validation sizes: " + str(len(training_data)) + "/" + str(len(val_data)))
print("Test size: " + str(len(test_data)))

Trainning / Validation sizes: 5063/2170
Test size: 50


In [153]:
class features_extractor_Model(nn.Module):

    def __init__(self, special_number):
        super(features_extractor_Model, self).__init__()
        # Feature Extractor part
        self.conv1 = nn.Conv1d(1, 5, 10)  # input is (x,y)
        self.pool = nn.MaxPool1d(10)
        self.conv2 = nn.Conv1d(5, 3, 10)
        self.conv3 = nn.Conv1d(3, 3, 3)
        self.fc1 = nn.Linear(3*special_number, 10)
        self.fc2 = nn.Linear(10, 10)
        
        #self.fc3 = nn.Linear(10, 3)

        #self.apply(self._init_weights)

    def _init_weights(self, module):
        #pdb.set_trace()
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=1.0)
            if module.bias is not None:
                module.bias.data.zero_()
        #torch.nn.init.xavier_uniform_(tensor, gain=1.0)
                
    def forward(self, x):
        # Feature Extractor part
        #pdb.set_trace()
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        #pdb.set_trace()
        x = x.view(-1, x.size(1)*x.size(2))
        x = F.relu(self.fc1(x))
        features = F.relu(self.fc2(x))
        
        return features

In [154]:
class classModel(nn.Module):

    def __init__(self):
        super(classModel, self).__init__()
        # Feature Extractor part
        self.fc1 = nn.Linear(10, 10)  # input is (x,y)
        self.fc2 = nn.Linear(10, 10)  # input is (x,y)
        self.fc3 = nn.Linear(10, 10)  # input is (x,y)
        self.fc4 = nn.Linear(10, 3)

        #self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=1.0)
            if module.bias is not None:
                module.bias.data.zero_()
                
    def forward(self, x):
        # Feature Extractor part
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        pred = F.softmax(self.fc4(x))
        #pred = self.fc1(x)
        
        return pred

In [155]:
class Trainer():
    def __init__(self, features_extractor, class_model, LEARNING_RATE, device="cpu"):
        self.features_extractor = features_extractor
        self.class_model = class_model
        self.criterion = nn.CrossEntropyLoss()
        self.LEARNING_RATE = LEARNING_RATE
        self.optimizer = optim.Adam(list(self.features_extractor.parameters()) + 
                   list(self.class_model.parameters()), lr=LEARNING_RATE)
        #self.features_extractor_optimizer = optim.Adam(self.features_extractor.parameters(), lr=LEARNING_RATE)
        #self.class_model_optimizer = optim.Adam(self.class_model.parameters(), lr=LEARNING_RATE)
        self.device = device
        
    def train(self, data_loader, eval_data_loader, EPOCHS):   
        self.features_extractor.zero_grad()
        self.features_extractor.train()
        self.class_model.zero_grad()
        self.class_model.train()
        
        ## Training Epochs
        for e in range(1, EPOCHS+1):
            epoch_bce_loss = 0
            epoch_acc = 0

            ## Mini-bacth Handling
            for X, y in data_loader:
                X, y = X.to(self.device), y.to(self.device)
                y = nn.functional.one_hot(y.long(), num_classes=3).float()
                self.optimizer.zero_grad()
                #self.features_extractor_optimizer.zero_grad()
                #self.class_model_optimizer.zero_grad()

                # Forward pass
                #pdb.set_trace()
                features = self.features_extractor(X)
                pred = self.class_model(features)

                # Loss calculations
                #pdb.set_trace()
                bce_loss = self.criterion(pred, y)                        
                loss = bce_loss
                acc = classification_acc(pred, y)

                # Back-propagation
                loss.backward()
                self.optimizer.step()
                #self.features_extractor_optimizer.step()
                #self.class_model_optimizer.step()

                # Metrics update
                epoch_bce_loss += bce_loss.item()
                epoch_acc += acc.item()

            print(f'Epoch {e+0:03}: | BCE Loss: {epoch_bce_loss/len(data_loader):.5f} | Train Acc: {epoch_acc/len(data_loader):.3f}')

            ## Evaluating on Target
            self.evaluate(eval_data_loader, data_type="Val")

    def evaluate(self, data_loader, data_type="Val"):
        test_loss = 0
        test_acc = 0
        self.features_extractor.eval()
        self.class_model.eval()
        batch_sizes = []
        
        ## Iterate over all dataset
        with torch.no_grad():
            for X_batch, y_batch in data_loader:
                #pdb.set_trace()
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                y_batch = nn.functional.one_hot(y_batch.long(), num_classes=3).float()
                
                features = self.features_extractor(X_batch)
                y_pred = self.class_model(features)
                loss = self.criterion(y_pred, y_batch)
                acc = classification_acc(y_pred, y_batch)
                
                batch_sizes.append(len(y_batch))
                
                test_loss += (len(y_batch) / batch_sizes[0]) * loss.item()
                test_acc += (len(y_batch) / batch_sizes[0]) * acc.item()
                
        nrml_factor = sum(batch_sizes) / batch_sizes[0]

        print(data_type + f' | Loss: {test_loss/nrml_factor:.5f} | Acc: {test_acc/nrml_factor:.3f}')

    def predict(self, data_loader):
        data = torch.empty((1,2))
        #pdb.set_trace()
        preds = torch.empty((0))
        y_real = torch.empty((0))
        self.features_extractor.eval()
        self.class_model.eval()
        
        ## Iterate over all dataset
        with torch.no_grad():
            for X_batch, y_batch in data_loader:
                #pdb.set_trace()
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                
                features = self.features_extractor(X_batch)
                y_pred = self.class_model(features)

                y_pred_tag = torch.softmax(y_pred, dim=1).argmax(dim=1)

                #data = torch.cat((data, X_batch))
                preds = torch.cat((preds, y_pred_tag))
                y_real = torch.cat((y_real, y_batch))

        return preds, y_real

In [156]:
torch.manual_seed(10)

## Train parameters
EPOCHS = 5
LEARNING_RATE = 0.003
device = 'cpu'

if X_training.shape[1] == 7133:
    special_number = 6
elif X_training.shape[1] == 16389:
    special_number = 16
elif X_training.shape[1] == 16075:
    special_number = 15
elif X_training.shape[1] == 2957:
    special_number = 2
elif X_training.shape[1] == 4930:
    special_number = 4
elif X_training.shape[1] == 2203:
    special_number = 1
elif X_training.shape[1] == 2361:
    special_number = 2
elif X_training.shape[1] == 2518:
    special_number = 2
elif X_training.shape[1] == 2204:
    special_number = 1
elif X_training.shape[1] == 2185:
    special_number = 1
elif X_training.shape[1] == 16658:
    special_number = 16
    
## Creating model
#input_dim = X_training.shape[1]
features_extractor = features_extractor_Model(special_number)
class_model = classModel()
features_extractor.to(device)
class_model.to(device)
#print(str(features_extractor)+"\n")
#print(str(class_model)+"\n")

## Training model
trainer = Trainer(features_extractor, class_model, LEARNING_RATE)
trainer.train(training_loader, val_loader, EPOCHS)

print()
trainer.evaluate(val_loader, "Val")
trainer.evaluate(test_loader, "Test")

Epoch 001: | BCE Loss: 0.85860 | Train Acc: 68.223
Val | Loss: 0.55316 | Acc: 100.000
Epoch 002: | BCE Loss: 0.55196 | Train Acc: 100.000
Val | Loss: 0.55148 | Acc: 100.000
Epoch 003: | BCE Loss: 0.55148 | Train Acc: 100.000
Val | Loss: 0.55146 | Acc: 100.000
Epoch 004: | BCE Loss: 0.55147 | Train Acc: 100.000
Val | Loss: 0.55146 | Acc: 100.000
Epoch 005: | BCE Loss: 0.55146 | Train Acc: 100.000
Val | Loss: 0.55145 | Acc: 100.000

Val | Loss: 0.55145 | Acc: 100.000
Test | Loss: 1.52933 | Acc: 0.000


In [157]:
trainer.predict(test_loader)

(tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
         2., 2., 2., 2., 2., 2., 2., 2., 2., 0., 0., 0., 0., 2., 2., 0., 0., 2.,
         0., 0., 0., 0., 0., 0., 2., 2., 2., 0., 2., 0., 0., 2.]),
 tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]))

In [158]:
y_test

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1], dtype=uint8)

In [159]:
y_prd = trainer.predict(test_loader)
y_prd = y_prd[0]

In [160]:
#y_test = [1, 0, 1, 1, 0, 1, 0, 0]
#y_prd = [1, 0, 1, 1, 0, 0, 1, 0]

from sklearn.metrics import confusion_matrix

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, y_prd)

print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[ 0  0  0]
 [15  0 35]
 [ 0  0  0]]
